In [ ]:
import pandas as pd
import numpy as np

In [ ]:
col_names = ['unit', 'cycle', 'op_set_1', 'op_set_2', 'op_set_3',
             'sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6',
             'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 
             'sensor13', 'sensor14', 'sensor15', 'sensor16', 'sensor17',
             'sensor18', 'sensor19', 'sensor20', 'sensor21']


data = pd.read_csv('train_FD001.txt',
                   delimiter=' ',
                   names=col_names,
                   index_col=False,
                   skipinitialspace=True
                   )

data.set_index(['unit', 'cycle'], inplace=True)

In [ ]:
data

op_set_1  op_set_2  op_set_3  ...  sensor19  sensor20  sensor21
unit cycle                                ...                              
1    1       -0.0007   -0.0004     100.0  ...     100.0     39.06   23.4190
     2        0.0019   -0.0003     100.0  ...     100.0     39.00   23.4236
     3       -0.0043    0.0003     100.0  ...     100.0     38.95   23.3442
     4        0.0007    0.0000     100.0  ...     100.0     38.88   23.3739
     5       -0.0019   -0.0002     100.0  ...     100.0     38.90   23.4044
...              ...       ...       ...  ...       ...       ...       ...
100  196     -0.0004   -0.0003     100.0  ...     100.0     38.49   22.9735
     197     -0.0016   -0.0005     100.0  ...     100.0     38.30   23.1594
     198      0.0004    0.0000     100.0  ...     100.0     38.44   22.9333
     199     -0.0011    0.0003     100.0  ...     100.0     38.29   23.0640
     200     -0.0032   -0.0005     100.0  ...     100.0     38.37   23.0522

[20631 rows x 24 columns]

In [ ]:
single = data.loc[1]
single.head()

,op_set_1,op_set_2,op_set_3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,sensor7,sensor8,sensor9,sensor10,sensor11,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
cycle,,,,,,,,,,,,,,,,,,,,,,,,
1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,2222.65,8341.91,1.02,42.02,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,2211.57,8303.96,1.02,42.20,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,9.02,175.71,1915.11,8001.42,0.94,36.69,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,2211.58,8303.96,1.02,41.96,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,9.03,175.05,1915.10,7993.23,0.94,36.89,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


In [ ]:
# To-Do - Build a time series window class that can produce a variable length 
# window with the same number of steps as there are number of run cycles.

# Reference link on tensorflow website here:
# https://www.tensorflow.org/tutorials/structured_data/time_series

# NOTES: The problem inherently with the data set provided for the Predictron
# architecture as it, is that it was designed to cycle through 2D representations 
# of mazes to find a singular known target. For the CMAPSS data where trying to
# cycle through a series of states until we arrive a single state that represents 
# the entire system. 

# When using time series windows like from the example above, it's built with the 
# idea that we'll be looking into the future N number of times, whereas with the 
# Predictron as applied to CMAPSS we're trying to determine how many remaining
# states are left. 

## The Predictron Model Class


In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: absl-py, grpcio, tensorflow-estimator, tensorboard, wrapt, protobuf, wheel, numpy, gast, h5py, opt-einsum, keras-preprocessing, scipy, six, astunparse, termcolor, google-pasta
Required-by: fancyimpute


In [ ]:
from tensorflow.keras.layers import BatchNormalization, Flatten, Conv1D, Dense
import tensorflow as tf
from tensorflow.keras.losses import MSE
import tensorflow.keras

class Predictron(tf.keras.Model):
  def __init__(self, dataset, RUL, max_depth):
    print(dataset)
    self.inputs = dataset
    self.targets = RUL
    self.learning_rate = 0.001
    self.size = 1 # dataset.shape[0] -- This was originally maze_size, which was originally 20
                  # this either needs to be 1 or the 24
    self.max_depth = max_depth
    self.batch_size = 5
    # self.run_shape = (self.batch_size, self.size, 24)

    self.rewards = None
    self.gammas = None
    self.lambdas = None
    self.values = None
    self.preturns = None
    self.lambda_preturns = None

  def build(self):
    self.build_model()
    self.build_loss()

  def iter_func(self, state):
    value_net = Flatten()(state)
    value_net = BatchNormalization()(value_net)
    value_net = Dense(1)(value_net)

    net = Conv1D(24, 3, input_shape=state.shape)(state)
    net = BatchNormalization()(net)
    flat_net = Flatten()(net)

    reward = Dense(24)(flat_net)
    reward = BatchNormalization()(reward)
    reward = Dense(1)(reward)

    gamma = Dense(24)(flat_net)
    gamma = BatchNormalization()(gamma)
    gamma = Dense(1)(gamma)

    lambda_net = Dense(24)(flat_net)
    lambda_net = BatchNormalization()(lambda_net)
    lambda_net = Dense(1)(lambda_net)

    net = Conv1D(24, 3, input_shape=state.shape)(net)
    net = BatchNormalization()(net)
    net = Conv1D(24, 3, input_shape=state.shape)(net)
    net = BatchNormalization()(net)

    return net, reward, gamma, lambda_net, value_net
  
  def build_model(self):
    # print(self.run_shape)
    state = Conv1D(24, 3, batch_size=batch_size)(self.inputs) # removed input_shape = self.run_shape
    state = BatchNormalization()(state)
    state = Conv1D(24, 3, batch_size=batch_size)(state) # removed input_shape = self.run_shape
    state = BatchNormalization()(state)
    print(state.shape)
    
    iter_template = tf.compat.v1.make_template('iterator', self.iter_func)

    rewards_arr = []
    gammas_arr = []
    lambdas_arr = []
    values_arr = []

    for k in range(self.max_depth):
      print(f'iteration: {k}')
      print(state)
      state, reward, gamma, lambda_, value = self.iter_func(state)
      rewards_arr.append(reward)
      gammas_arr.append(gamma)
      lambdas_arr.append(lambda_)
      values_arr.append(value)
      print(lambdas_arr)

    _, _, _, _, value = self.iter_func(state)
    values_arr.append(value)

    bs = tf.shape(self.inputs)[0]
    print(f'BS: {bs}')

    # I think the negative output size is happening here because we're still defining
    # self.size, which I don't think is userful for the C-MAPSS dataset since it's
    # on dimensional. So far options for self.size are:
    # 1. remove it
    # 2. set it to the feature size of the dataset (I think we've done that already)
    # 3. set it dynamically to the current state shape?
    #
    # Update - The negative output size isn't happening here, but it might need to be
    # rewritten to accomodate 1D data
    self.rewards = tf.stack(rewards_arr, axis=1)
    self.rewards = tf.reshape(self.rewards, [bs, self.max_depth, self.size]) 
    self.rewards = tf.concat(values=[tf.zeros(shape=[bs, 1, self.size], dtype=tf.float32),
                                      self.rewards], axis=1, name='rewards')
    self.gammas = tf.stack(gammas_arr, axis=1)
    self.gammas = tf.reshape(self.gammas, [bs, self.max_depth, self.size])
    self.gammas = tf.concat_v2(values=[tf.ones(shape=[bs, 1, self.size], dtype=tf.float32), self.gammas],
                              axis=1, name='gammas')
    self.lambdas = tf.stack(lambdas_arr, axis=1)
    self.lambdas = tf.reshape(self.lambdas, [-1, self.max_depth, self.size])
    self.values = tf.stack(values_arr, axis=1)
    self.values = tf.reshape(self.values, [-1, (self.max_depth +1), self.size])

    self.build_preturns()
    self.build_lambda_preturns()

  def build_preturns(self):
    g_preturns = []
    for k in range(self.max_depth, -1, -1):
      g_k = self.values[:, k, :]
      for kk in range(k, 0, -1):
        g_k = self.rewards[:, kk, :] + self.gammas[:, kk, :] * g_k
      g_preturns.append(g_k)
    g_preturns = g_preturns[::-1]
    self.preturns = tf.stack(g_preturns, axis=1, name='preturns')
    self.g_preturns = tf.reshape(self.g_preturns, [-1, (self.max_depth + 1), self.size])

  def build_lambda_preturns(self):
    g_k = self.values[:, -1, :]
    for k in range(self.max_depth -1, -1, -1):
      g_k = (1 - self.lambdas[:, k, :]) * self.values[:, k, :] + self.lambdas[:, k, :] * (self.rewards[:, k + 1, :] + self.gammas[:, k + 1, :] * g_k)
    self.g_lambda_preturns = g_k

  def build_loss(self):
    #need to rebuild this to a loop through each run cycle

    self.targets_expanded = tf.expand_dims(self.target, 1) # 
    self.targets_tiled = tf.tile(self.targets_expanded, [1, self.max_depth +1, 1])
    self.loss_preturns = MSE(self.g_preturns, self.targets_tiled)
    tf.compat.v1.losses.add_loss(self.loss_preturns)
    tf.summary.scalar('Loss Preturns', self.loss_preturns)
    self.loss_lambda_preturns - MSE(self.g_lambda_preturns, self.targets)
    tf.compat.v1.losses.add_loss('Loss Lambda Preturns', self.loss_lambda_preturns)
    self.total_loss = losses.get_total_loss(name='Total Loss')


## The Training Module

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import six.moves.queue as queue
from six.moves import range
import datetime
import logging
import os
import glob
import threading
import time

import numpy as np
import tensorflow as tf


# FLAGS = tf.app.flags.FLAGS

# tf.flags.DEFINE_string('train_dir', './ckpts/predictron_train',
#                        'dir to save checkpoints and TB logs')
# tf.flags.DEFINE_integer('max_steps', 10000000, 'num of batches')
# tf.flags.DEFINE_float('learning_rate', 1e-3, 'learning rate')

# tf.flags.DEFINE_integer('batch_size', 128, 'batch size')
# tf.flags.DEFINE_integer('maze_size', 20, 'size of maze (square)')
# tf.flags.DEFINE_float('maze_density', 0.3, 'Maze density')
# tf.flags.DEFINE_integer('max_depth', 16, 'maximum model depth')
# tf.flags.DEFINE_float('max_grad_norm', 10., 'clip grad norm into this value')
# tf.flags.DEFINE_boolean('log_device_placement', False,
#                         """Whether to log device placement.""")
# tf.flags.DEFINE_integer('num_threads', 10, 'num of threads used to generate mazes.')

train_dir = '/'
max_steps = 10000
learning_rate = 0.0001
batch_size = 32
# define 'size' after dataframe import
max_depth = 10
max_grad_norm = 10.
log_device_placement = False
num_threads = 10


# logging.basicConfig()
# logger = logging.getLogger('training')
# logger.setLevel(logging.INFO)


def train():
  # config = FLAGS

  # global_step = tf.get_variable(
  #   'global_step', [],
  #   initializer=tf.constant_initializer(0), trainable=False)

  sensor_data_ph = tf.Variable(tf.ones((1, 100, 24, 1), tf.float32))
  labels_ph = tf.Variable(tf.zeros((24), tf.float32))

  model = Predictron(sensor_data_ph, labels_ph, max_depth)
  model.build()

  loss = model.total_loss
  loss_preturns = model.loss_preturns
  loss_lambda_preturns = model.loss_lambda_preturns

  opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  grad_vars = opt.get_gradients(loss, tf.trainable_variables())
  grads, vars = zip(*grad_vars)
  grads_clipped, _ = tf.clip_by_global_norm(grads, FLAGS.max_grad_norm)
  grad_vars = zip(grads_clipped, vars)
  apply_gradient_op = opt.apply_gradients(grad_vars, global_step=global_step)

  # update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  # update_op = tf.group(*update_ops)
  # # Group all updates to into a single train op.
  # train_op = tf.group(apply_gradient_op, update_op)

  # init = tf.global_variables_initializer()
  # sess = tf.Session()
  # sess.run(init)

  # saver = tf.train.Saver(tf.global_variables())
  # tf.train.start_queue_runners(sess=sess)

  train_dir = os.path.join(FLAGS.train_dir, 'max_steps_{}'.format(FLAGS.max_depth))
  # summary_merged = tf.summary.merge_all()
  # summary_writer = tf.summary.FileWriter(train_dir, sess.graph)

  col_names = ['unit', 'cycle', 'op_set_1', 'op_set_2', 'op_set_3',
              'sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6',
              'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 
              'sensor13', 'sensor14', 'sensor15', 'sensor16', 'sensor17',
              'sensor18', 'sensor19', 'sensor20', 'sensor21']
  
  data_master = pd.DataFrame(columns=col_names)

  data_master.set_index(['unit', 'cycle'], inplace=True)
  
  def read_cmapss(file):
    df = pd.read_csv(file,
                      delimiter=' ',
                      names=col_names,
                      index_col=False,
                      skipinitialspace=True
                      )
    df.set_index(['unit', 'cycle'], inplace=True)
    # for unit in df:
    #   sensor_data = unit.to_numpy
    #   labels = sensor_data.shape[1]
    #   frame_queue.put((sensor_data, labels))
    data_master.append(df)

  def cmapss_queue():
    dat_dir = '/'
    for file in glob.glob(os.path.join(dat_dir, '.txt')):
      read_cmapss(file)

  # for thread_i in range(num_threads):
  #   t = threading.Thread(target=cmapss_queue)
  #   t.start()

  for step in range(max_steps):
    print("I am at Max_steps in training")
    start_time = time.time()
    sensor_data_np, labels_np = frame_queue.get()

    _, loss_value, loss_preturns_val, loss_lambda_preturns_val, summary_str = sess.run(
      [train_op, loss, loss_preturns, loss_lambda_preturns, summary_merged],
      feed_dict={
        sensor_data_ph: sensor_data_np,
        labels_ph: labels_np
      })
    duration = time.time() - start_time

    assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

    if step % 10 == 0:
      num_examples_per_step = batch_size
      examples_per_sec = num_examples_per_step / duration
      sec_per_batch = duration

      format_str = (
        '%s: step %d, loss = %.4f, loss_preturns = %.4f, loss_lambda_preturns = %.4f (%.1f examples/sec; %.3f '
        'sec/batch)')
      logger.info(format_str % (datetime.datetime.now(), step, loss_value, loss_preturns_val, loss_lambda_preturns_val,
                                examples_per_sec, sec_per_batch))
      print(f"{datetime.datetime.now()}, step {step}, loss = {loss_value}, loss_preturns = {loss_preturns_val}, loss_lambda_preturns = {loss_lambda_preturns_val}")

    # if step % 100 == 0:
    #   summary_writer.add_summary(summary_str, step)

    # Save the model checkpoint periodically.
    # if step % 1000 == 0 or (step + 1) == max_steps:
    #   checkpoint_path = os.path.join(train_dir, 'model.ckpt')
    #   saver.save(sess, checkpoint_path, global_step=step)


In [ ]:
train()

<tf.Variable 'Variable:0' shape=(1, 100, 24, 1) dtype=float32, numpy=
array([[[[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        ...,

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]]], dtype=float32)>
(1, 100, 20, 24)
iteration: 0
tf.Tensor(
[[[[-0.30973434  0.4164496   0.02719295 ... -0.12441047 -0.08611278
    -0.03347215]
   [-0.30973434  0.4164496   0.02719295 ... -0.12441047 -0.08611278
    -0.03347215]
   [-0.30973434  0.4164496   0.02719295 ... -0.12441

InvalidArgumentError: ignored